# Things used
#### https://www.thepythoncode.com/article/finetuning-bert-using-huggingface-transformers-python
#### https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b

In [1]:
!nvidia-smi

Wed Feb  2 19:33:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd "/content/drive/MyDrive/WPI/Junior/IQP/Machine Models/data/v9"

/content/drive/MyDrive/WPI/Junior/IQP/Machine Models/data/v9


In [4]:
# ! unzip followers.zip

In [5]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 13.6 MB/s 
     |████████████████████████████████| 6.8 MB 54.2 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
     |████████████████████████████████| 895 kB 62.0 MB/s 
     |████████████████████████████████| 596 kB 66.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import json
import torch
import numpy as np
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [7]:
# Get device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
# import os
# assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [8]:
# Json path
json_path = 'bad_user_tweets_with_seed_v5.json'
json_path_good = 'clean_tweets_v9.json'

In [9]:
# Open json file
bad_user_tweets_file = open(json_path, 'r')
# Load json file
bad_user_tweets = json.load(bad_user_tweets_file)
bad_user_tweets = bad_user_tweets['IWTSeller']

# Open json file
good_user_tweets_file = open(json_path_good, 'r')
# Load json file
good_user_tweets = json.load(good_user_tweets_file)
good_user_tweets = good_user_tweets['tweets']

In [10]:
#remove punctuation
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

table = str.maketrans('', '', string.punctuation)

# Remove stop words
from nltk.corpus import stopwords

stop = stopwords.words('english')

# Lemmatize all words
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()	

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [11]:
bad_tweet_text_list = []
for tweet in bad_user_tweets:
    tweet_text = tweet['tweet']
    tweet_text = ' '.join([word for word in tweet_text.lower().split() if word not in stop]) #remove stopwords
    tweet_text = tweet_text.translate(table) #remove punc.
    tweet_text = ''.join(c for c in tweet_text if not c.isdigit()) #remove numbers
    tweet_text = tweet_text.replace('  ', ' ') #remove double spaces
    tweet_text = ' '.join([lemmatizer.lemmatize(word) for word in tweet_text.split()]) #lemminize
    bad_tweet_text_list.append(tweet_text)

In [12]:
good_tweet_text_list = []
# for user in good_user_tweets:
# 	try:
# 		for tweet in good_user_tweets[user]['data']:
for tweet in good_user_tweets:
			tweet_text = tweet['text']
			tweet_text = ' '.join([word for word in tweet_text.lower().split() if word not in stop]) #remove stopwords
			tweet_text = tweet_text.translate(table) #remove punc.
			tweet_text = ''.join(c for c in tweet_text if not c.isdigit()) #remove numbers
			tweet_text = tweet_text.replace('  ', ' ') #remove double spaces
			tweet_text = ' '.join([lemmatizer.lemmatize(word) for word in tweet_text.split()]) #lemminize
			good_tweet_text_list.append(tweet_text)

In [13]:
# Combine bad and good tweets
tweet_combined_ground_truth = bad_tweet_text_list + good_tweet_text_list
print(len(tweet_combined_ground_truth))
print(tweet_combined_ground_truth[-1])


397
httpgooglrueeg


In [14]:
# Create the ground truth labels
bad_labels = [1] * len(bad_tweet_text_list)
good_labels = [0] * len(good_tweet_text_list)
# Combine the labels
labels = bad_labels + good_labels

In [15]:
print(len(labels))

397


In [16]:
model_name = "bert-base-uncased"
# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [17]:
from sklearn.model_selection import train_test_split
x_train, y_train, x_test, y_test = train_test_split(tweet_combined_ground_truth, labels, test_size=0.2, random_state=1, stratify=labels)

In [18]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

['rt christinevans today… maskless sfpd officer fails remove dpw worker shoved guy instead continues with glo…', 'damnarbor ill allow it also look hideous s era apartment ows see blah be bad response ows association absurd good inbetween', 'rt projectlincoln free signup meet donald trump know do… httpstcoltwmpwjxk', 'icicibankcare close bank account india im singapore give account closing request singapore branch', 'new acquisition fantastic pair art deco bronze amp ivory figure artdeco bronze ivory httpstcomiokrgjbrw httpstconqyklqfqal', 'אהוד ברק אולי תתחיל לסתום את הפה שלך אתה בן אדם המייצר שינאה ומתוסכל אל דאגה נסתדר בלעדיך', 'selling beautiful piano anyone interested loved get little playing attention sob mahogany iron frame ivory key stamped fine condition regularly tuned purchased httpstcopdqaskcbt', 'bitch too try', 'congrats httpstcotsufc', 'leleadani uomo vero oltre che persona professionale e preparata', 'started rehearsal mario oharas stageshow mario able attendthe reading 

In [19]:
train_encodings = tokenizer(x_train, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(y_train, truncation=True, padding=True, max_length=max_length)

In [20]:
# print(train_encodings)
print(type(train_encodings))
print(len(y_train))

<class 'transformers.tokenization_utils_base.BatchEncoding'>
80


In [21]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # print(item)
        # print("Number of encodings: " + str(len(self.encodings)))
        # print(str(idx) + ' ' + str(len(self.labels)))
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [22]:
train_dataset = Dataset(train_encodings, x_test)
test_dataset = Dataset(test_encodings, y_test)

In [23]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [24]:
from sklearn.metrics import classification_report

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    classification = classification_report(labels, pred, output_dict=True)

    return classification

In [25]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=100,              # total number of training epochs
    per_device_train_batch_size=128,  # batch size per device during training
    per_device_eval_batch_size=128,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=25,               # log & save weights each logging_steps
    save_steps=25,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [26]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,        # training dataset
    eval_dataset=test_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,  # the callback that computes metrics of interest
)


In [27]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 317
  Num Epochs = 100
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 300


Step,Training Loss,Validation Loss,0,1,Accuracy,Macro avg,Weighted avg
25,0.650100,0.606215,"{'precision': 0.8703703703703703, 'recall': 1.0, 'f1-score': 0.9306930693069307, 'support': 47}","{'precision': 1.0, 'recall': 0.7878787878787878, 'f1-score': 0.8813559322033898, 'support': 33}",0.912500,"{'precision': 0.9351851851851851, 'recall': 0.8939393939393939, 'f1-score': 0.9060245007551603, 'support': 80}","{'precision': 0.9238425925925926, 'recall': 0.9125, 'f1-score': 0.9103415002517201, 'support': 80}"
50,0.504700,0.364993,"{'precision': 1.0, 'recall': 0.9787234042553191, 'f1-score': 0.989247311827957, 'support': 47}","{'precision': 0.9705882352941176, 'recall': 1.0, 'f1-score': 0.9850746268656716, 'support': 33}",0.987500,"{'precision': 0.9852941176470589, 'recall': 0.9893617021276595, 'f1-score': 0.9871609693468143, 'support': 80}","{'precision': 0.9878676470588236, 'recall': 0.9875, 'f1-score': 0.9875260792810142, 'support': 80}"
75,0.269100,0.164919,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 47}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}",1.000000,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}"
100,0.099000,0.032901,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 47}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}",1.000000,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}"
125,0.015800,0.003742,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 47}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}",1.000000,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}"
150,0.003100,0.001373,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 47}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}",1.000000,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}"
175,0.001500,0.000795,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 47}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}",1.000000,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}"
200,0.001000,0.000530,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 47}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}",1.000000,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}"
225,0.000700,0.000383,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 47}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}",1.000000,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}"
250,0.000500,0.000290,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 47}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}",1.000000,"{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}","{'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 80}"


***** Running Evaluation *****
  Num examples = 80
  Batch size = 128
Trainer is attempting to log a value of "{'precision': 0.8703703703703703, 'recall': 1.0, 'f1-score': 0.9306930693069307, 'support': 47}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 1.0, 'recall': 0.7878787878787878, 'f1-score': 0.8813559322033898, 'support': 33}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9351851851851851, 'recall': 0.8939393939393939, 'f1-score': 0.9060245007551603, 'support': 80}" of type <class 'dict'> for key "eval/macro avg" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'pre

TrainOutput(global_step=300, training_loss=0.12884930418493848, metrics={'train_runtime': 331.1029, 'train_samples_per_second': 95.741, 'train_steps_per_second': 0.906, 'total_flos': 1400963592036000.0, 'train_loss': 0.12884930418493848, 'epoch': 100.0})

In [28]:
# import gensim.downloader as api
# wv = api.load('word2vec-google-news-300')

In [29]:
# tweet_text_vectors = []
# for tweet in tweet_text_list:
# 	for word in tweet.split():
# 			try:
# 				tweet_text_vectors.append(np.asarray(wv[word]).astype('float32'))
# 			except:
# 				print(f'Word {word} not in vocabulary')